# GuildedChatExporter v2.2

New in 2.2 - WIP for Guilded's death: media fixing

Export your stuff before it's gone. This will give you HTML, media files, though more importantly, the RAW JSON files in case your data needs to be re-processed in the future.

How this works: Log in to Google (sorry) to get this running, and then for each relevant step, click the Play/Run arrow on the left of each cell/block that you see while reading and following the instructions.

If you want to view the files fetched, you may click the folder icon on the left sidebar and also download them individually. Downloading everything comes at the end.

*   **You absolutely MUST run 0.x steps.**
*   1.x steps are for servers. Can be skipped.
*   2.x steps are for DMs.

Any issues? Contact Fatih here or use the Github tracker. [Discord Server](https://discord.com/invite/Cy27FNfQtc)

Obviously I'm not liable for anyone else's usage of these scripts, especially for the DMs deleter. Read what the code does or get someone else to do it for you.

In [ ]:
# @title 0.0 Basic Setup

# @markdown 1.   **Log on to Guilded on your web browser, not client, and focus on that tab.**

# @markdown If based on **Firefox**, press `SHIFT + F9`.
# @markdown
# @markdown If on **Chrome**, press `F12` and find the "Application" tab.
# @markdown
# @markdown **We're here to grab your login, so click the Cookies entry on the sidebar, and then the guilded.gg url.**
# @markdown
# @markdown 2. Find the `hmac_signed_session` key at the bottom. Copy the value, it is very long. Paste it here.

#!apt-get install -y p7zip-full
import requests
import json
import re
import os
import shutil
import urllib.parse
import ipywidgets as widgets
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.colab import files
import urllib.parse
from urllib.parse import urlparse
!pip install emoji
import emoji

USER_TOKEN = "XXXX.YYYY.ZZZZ" # @param {type:"string"}
# @markdown Do NOT share this value with anyone as they will be able to access your account without a password.

SAVE_DIRECTORY = "/content/guildedchatexporter/" # @param {type:"string"}
if not os.path.exists(SAVE_DIRECTORY):
  os.makedirs(SAVE_DIRECTORY)

cookies = {
    "authenticated": "true",
    "hmac_signed_session": USER_TOKEN,
}

# @markdown Google Colab gives you over 100GB of temporary space to work with, which should be way more than enough to store and then download your stuff.
# @markdown
# @markdown  You can also connect your Google Drive instead and copy the stuff to there at the end, though if you just want to download your files immediately without using Google Drive, skip the optional step.
# --------------------------------------------------- #

guilded = "https://www.guilded.gg/api"
def fetch(endpoint, params=None):
    url = f"{guilded}/{endpoint}"
    response = requests.get(url, params=params, cookies=cookies)
    if response.status_code == 200:
      return unshid_cdn(response.json())
    else:
        raise Exception(f"Error fetching data: {response.status_code} {response.text}")
        print("Did you correctly input your token?")

def unshid_cdn(data):
  def fix_url(url):
    # This regex matches the old s3 domain and captures everything after it (including query params)
    match = re.match(r"^https://s3-us-west-2\.amazonaws\.com/www\.guilded\.gg/(.*)$", url)
    if match:
        # Replace domain but keep the full path and query string after the base
        return f"https://cdn.gldcdn.com/{match.group(1)}"
    else:
        return url

  def fix_dict(d):
    for key, value in d.items():
      if isinstance(value, dict):
        fix_dict(value)
      elif isinstance(value, list):
        for item in value:
          if isinstance(item, dict):
            fix_dict(item)
          elif isinstance(item, str):
            d[key] = fix_url(item)
      elif isinstance(value, str):
        d[key] = fix_url(value)

  fix_dict(data)
  return data

def fetch_user(user_id):
    data = fetch(f"users/{user_id}")
    return data["user"]

def fetch_channel(channel_id):
    data = fetch("content/route/metadata", params={"route": f"//channels/{channel_id}/chat"})
    return data["metadata"]["channel"]

def fetch_servers():
    data = fetch("me", params={"isLogin": "false", "v2": "true"})
    return data['teams']

def fetch_members():
    data = fetch(f"teams/{SERVER_ID}/members")
    return data

def get_groups():
    return fetch(f"teams/{SERVER_ID}/groups")

def get_groups():
    return fetch(f"teams/{SERVER_ID}/groups")

def get_channels():
    return fetch(f"teams/{SERVER_ID}/channels")

def fetch_dms():
    # Fetch DM channels directly from the channels endpoint
    data = fetch(f"users/{USER_ID}/channels")
    return data

def fetch_myself():
    return fetch("me", params={"isLogin": "false", "v2": "true"})

def fetch_info():
    return fetch(f"teams/{SERVER_ID}/info")

def sanitize_filename(filename):
    filename = re.sub(r'[<>:"/\\|?*]', '_', filename)
    filename = filename.rstrip(' _')
    if not filename:
        filename = 'untitled'
    return filename[:250]

from IPython.display import clear_output
clear_output()

print("Please select the server you wish to archive in the dropdown.")
server_dict = {f"{server['name']} ({server['id']})": server["id"] for server in fetch_servers()}
server_dict[''] = None
server_dropdown = widgets.Dropdown(options=list(server_dict.keys()), value='', description='Servers:',)
display(server_dropdown)

def on_server_select(change):
    global SERVER_ID
    global SERVER_NAME
    selected_server_name = change["new"]
    SERVER_ID = server_dict[selected_server_name]
    SERVER_NAME = selected_server_name
    print(f"Selected server: {selected_server_name}")
    print("You can go to the next step, or you can go straight for DMs")

server_dropdown.observe(on_server_select, names="value")


# 0.1 AWS Setter

Guilded has gotten more complicated and direct image links don't work as simply as they should anymore.

Just like in 0.0, you must fetch this info yourself from Guilded, so focus to that tab.

*   If on **Firefox**, press `Ctrl+Shift+K`
*   If on **Chrome**, press `Control+Shift+J`
*   ...or find the Console tab.

This will pull up the **Console**. It will bring up a scary warning saying not to paste anything in there, but that's exactly what we're gonna do.

We're going to fetch a random image that hopefully contains the parameters we need to load all media from now on.

Paste this code into the console:

```
(function() { // Hopefully you can see what this does.
  const types = document.querySelectorAll('img, video, audio, source');
  for (const el of types) {
    const url = el.src || el.currentSrc || el.getAttribute('src');
    if (url && url.includes('Expires=') && url.includes('Expires=') && url.includes('Signature=') && url.includes('Key-Pair-Id=')) {
      console.log("%c▼▼▼ RIGHT CLICK, COPY THIS LINK, AND PASTE IT TO EXPORTER ▼▼▼", "font-weight: bold; font-size: 20px;");
      return url;
    }
  }
  console.log("Didn't find an image for authenticating, try jumping around and loading more images");
  return null;
})();
```

As it will tell you, copy the resulting link that appears. If one doesn't load... (WIP idk contact us, because I don't know in what scenario it doesn't)

In [ ]:
# @markdown Paste the link here, then click Run on this cell.

consoleurl = "https://cdn.gldcdn.com/example/filename?LotsOfCrap" # @param {type:"string"}

# @markdown This now hopefully sets the parameters needed to download media for this session. You might have to do this every time you start the exporter all over again.

from urllib.parse import urlparse, parse_qs

def parse_aws(consoleurl):
    awsed = urlparse(consoleurl)
    media_params = parse_qs(awsed.query)
    aws_params = {}
    for key in ["Expires", "Policy", "Signature", "Key-Pair-Id"]:
        if key in media_params:
            aws_params[key] = media_params[key][0]
    return aws_params

AWS_PARAMS = parse_aws(consoleurl)
print(AWS_PARAMS)
print("If you see Expires, Policy, Signature, Key-Pair-Id, go to the next step!")

In [ ]:
# @title (Required) HTML File Setup
# @markdown Has the base HTML and CSS for output. Make sure to run this so that it's applied.

# @markdown To-do: the font file is not future-proof when Guilded dies
## HTML Template

CSS = """
@font-face {
  font-family: "Builder Sans";
  src: url("https://www.guilded.gg/fonts/BuilderSans-Regular.woff2") format('woff2');
}

body {
  font-family: 'Builder Sans', Tahoma, sans-serif;
  color: white;
  background-color: #373943;
}

#chat-log {
  background-color: #373943;
  padding: 16px;
  display: flex;
  flex-direction: column;
  gap: 8px;
}

.message {
  color: #ececee;
  font-size: 15px;
  line-height: 145%;
  font-weight: normal;
  font-family: 'Builder Sans', Tahoma, sans-serif;
  margin-top: 0px;
  min-height: 44px;
  padding: 8px 12px;
  display: flex;
  align-content: flex-start;
  gap: 8px;
  transition: background-color 140ms ease;
  position: relative;
  contain: layout;
  max-width: 80%;
}

.message:hover {
  background-color: #31333c;
}

.created-at {
  margin-right: 8px;
  height: 11px;
  color: #a3a3ac;
  font-size: 13px;
  line-height: 120%;
  font-weight: normal;
  white-space: nowrap;
  pointer-events: auto;
}

.created-by {
  font-size: 15px;
  line-height: 145%;
  font-weight: 700;
  white-space: nowrap;
  pointer-events: auto;
}

.pfp {
  width: 40px;
  height: 40px;
  border-radius: 50%;
  overflow: hidden;
  vertical-align: middle;
  flex-shrink: 0;
  margin-right: 12px;
  pointer-events: auto;
}

.emote {
  height: 1.3em;
  vertical-align: middle;
}

.content {
  margin: 0;
  word-break: break-all;
}

.content img,
.content video {
  max-height: 320px;
  max-width: 360px;
  object-fit: contain;
}

.file-upload-container {
  display: flex;
  align-items: center;
  background-color: #32343d;
  padding: 10px;
  border: 1px #a3a3ac solid;
  border-radius: 5px;
	color: #fef0b9;
  margin-top: 12px;
  margin-bottom: -40px;
}

.file-icon {
  margin-right: 10px;
  background-size: contain;
}

.file-info {
    flex-grow: 1;
}

.file-name {
    font-weight: bold;
    text-decoration: none;
    color: #fef0b9;
}

.file-size {
    color: #666;
    font-size: 0.9em;
}

.download-icon {
    margin-left: 10px;
    color: #666;
}

.icon {
    width: 22px;
    height: 22px;
    background-image: url('data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABIAAAAWCAMAAAD6gTxzAAAAk1BMVEUAAADt7e3u7u7w8PDy8vLo6PPp6fDt7e3p6e/r6/Dt7e3t7e3s7Ozt7e3t7e3s7O/s7O/t7e3t7e3s7O/r6+7r6+7t7e/t7e3r6+3r6+3s7O7s7O/s7O7s7O7t7e7s7O7s7O/t7e7s7O7s7O7s7O7s7O7t7e7r6+7s7O7s7O/s7O7s7O7s7O7s7O7s7O7s7O7////r1//TAAAAL3RSTlMADg8RExYiKi8zODlERUhQUVVWX2Znb3FzdHh8h5KZoarBx8jMzs/Q19nq7vD6/rP6L04AAAABYktHRDCu3C3kAAAAiklEQVQY04XR2RKCMAyF4eCCK+KOoOCKu+T9305KT0MHmOG/6nyT5qKlyYuLfqlP6H5ZqzbHjCPQd4tDwsaElnyACc25E/K+SqTNoqHrJuquRTq/pP40bxZn55LQ6V2j3acgb4E8okDTA5s5FXK6yBGyAw1Gqp5NeteqZaphV4Xw0NL1RuY7TM/xH0oeGJMDGiugAAAAAElFTkSuQmCC');
    background-repeat: no-repeat;
}

"""

HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>#{title}</title>
    <style>
        {CSS}
    </style>
</head>
<body>
    <div id="chat-log">
        <p>#{title}</p>
        <p>{topic}</p>
        <hr>
        {messages}
    </div>
</body>
</html>
"""

## Message Template
MESSAGE_TEMPLATE = """
<div class="message">
    <img class="pfp" src="{avatar_url}">
    <div>
        <span class="created-by">{created_by}</span>
        <span class="created-at">({created_at})</span>
        <p class="content">{content}</p>
    </div>
</div>
"""

print("HTML ready to print")


In [ ]:
# @title 1.0 Save Basic Data about Server and Users
if 'SERVER_ID' not in globals():
    print("You forgot to choose a server. Go back and select one from the dropdown.")
else:
    server_dir = os.path.join(SAVE_DIRECTORY, SERVER_NAME)
    os.makedirs(server_dir, exist_ok=True)


verbose = True # @param {type:"boolean"}
get_audits = False # @param {type:"boolean"}
# @markdown You can try to get audit logs for the server, but it'll fail if you're not authorized to see them.

# @markdown **If you know the names of certain [deleted user]s on the server and don't want their messages to appear as such, note their IDs and edit the members.json found in the server save directory. All the deleted members are stored in there, so just replace their name with the one you wish for.**

def download_file(url, filepath):
    !wget -q -nc -O "$filepath" "$url"
mc = 0
members_data = fetch_members()
members = members_data.get('members', [])

overview = fetch(f"teams/{SERVER_ID}/overview")
with open(os.path.join(server_dir, 'overview.json'), 'w') as f:
    f.write(json.dumps(overview, indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/overview.json")


all_audit_logs = []
before_date = None
if get_audits == True:
    print("Getting audit logs...")
    while True:
        params = {"maxItems": 50}
        if before_date:
            params["beforeDate"] = before_date
        response = requests.post(f"https://www.guilded.gg/api/teams/{SERVER_ID}/auditlogs", json=params, cookies=cookies)
        response.raise_for_status()
        audit_logs = response.json()
        new_logs = audit_logs["logs"]
        all_audit_logs.extend(new_logs)
        if len(new_logs) < 50:
            break
        !sleep 1
        before_date = new_logs[-1]["createdAt"]
    with open(os.path.join(server_dir, 'auditlogs.json'), 'w') as f:
        json.dump(all_audit_logs, f, indent=4)
    print(f"Saved {len(all_audit_logs)} audit logs to {os.path.join(server_dir, 'auditlogs.json')}")



for member in members:
    member_id = member["id"]
    member_name = member["name"]
    member_dir = os.path.join(server_dir, "members", member_name)
    os.makedirs(member_dir, exist_ok=True)
    user_data = fetch_user(member_id)

    with open(os.path.join(member_dir, f"{member_id}.json"), 'w') as f:
        f.write(json.dumps(user_data, indent=4))
    json_string = json.dumps(user_data)

    cdn_links = re.findall(r"https://cdn\.gldcdn\.com/[^\"]+", json_string)
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(download_file, url, os.path.join(member_dir, url.split("/")[-1].split("?")[0])) for url in cdn_links]

    mc += 1
    if verbose:
      print(f"{str(mc)}/" + str(len(members)) + f" {member_name} ({member_id})")



with open(os.path.join(server_dir, 'members.json'), 'w') as f:
    f.write(json.dumps(fetch_members(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/members.json")
with open(os.path.join(server_dir, 'info.json'), 'w') as f:
    infourls = []
    info = fetch_info()
    f.write(json.dumps(info, indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/info.json")
    if 'profilePicture' in info['team']:
        infourls.append(info['team']['profilePicture'])
    if 'teamDashImage' in info['team']:
        infourls.append(info['team']['teamDashImage'])
    if 'homeBannerImageLg' not in info['team']:
        if 'homeBannerImageMd' in info['team']:
            infourls.append(info['team']['homeBannerImageMd'])
        if 'homeBannerImageSm' in info['team']:
            infourls.append(info['team']['homeBannerImageSm'])
    else:
        infourls.append(info['team']['homeBannerImageLg'])
    for url in infourls:
        if verbose:
            print(f"checking: {type(url)}, {url}")
        if isinstance(url, bytes):
            print(f"this is apparently a BYTE URL: {url}")
            continue

        try:
            parsed_url = urlparse(url)
            basename = os.path.basename(parsed_url.path)
            full_path = os.path.join(server_dir, basename)
            if verbose:
                print(f"Basename: {type(basename)}, {basename}")
                print(f"Full path: {type(full_path)}, {full_path}")
            download_file(url, full_path)
            print(f"Successful: {url}")
        except Exception as e:
            print(f"Error with URL {url}: {str(e)}")

with open(os.path.join(server_dir, 'groups.json'), 'w') as f:
    f.write(json.dumps(get_groups(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/groups.json")
with open(os.path.join(server_dir, 'channels.json'), 'w') as f:
    f.write(json.dumps(get_channels(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/channels.json")

print("Done getting some fundamental stuff, go to the next step")

In [ ]:
# @title 1.1 Save All Emoji
from google.colab import files
emotes_data = fetch(f"teams/{SERVER_ID}/customReactions")

verbose = True # @param {type:"boolean"}
emotes_dir = os.path.join(server_dir, "emotes")
download_immediately = False # @param {type:"boolean"}

def fetch_emotes():
    os.makedirs(emotes_dir, exist_ok=True)

    with open(os.path.join(server_dir, 'emotes.json'), 'w') as f:
        f.write(json.dumps(emotes_data, separators=(',', ':')))
    print("Saved emotes.json")

    ec = 0
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        downloaded = set()  # need to check if we already got it cuz dupes, waste less time
        for reaction in emotes_data.get('reactions', []):
            for image_type in ["png", "webp", "apng"]:
                url = reaction.get(image_type)
                if url and url not in downloaded:
                    # Append the AWS S3 parameters to the URL
                    parsed_url = urlparse(url)
                    query_params = urllib.parse.urlencode(AWS_PARAMS)
                    modified_url = f"{url}?{query_params}"

                    filename = modified_url.split("/")[-1].split("?")[0]
                    file_extension = filename.split(".")[-1]
                    filepath = os.path.join(emotes_dir, f"{reaction['id']}-{reaction['name']}.{filename.split('.')[-1]}")

                    futures.append(executor.submit(download_emote, modified_url, filepath))
                    downloaded.add(url)
                    ec += 1
                    if verbose:
                      print(f"{ec}/{str(len(emotes_data.get('reactions', [])))} " + reaction["name"])

        for future in as_completed(futures):
            future.result()

        print("All done with server emotes!")

def download_emote(url, filepath):
    # Use the requests library to download the emote with the modified URL
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
    except requests.exceptions.RequestException as e:
        print(f"Error downloading emote from {url}: {e}")


fetch_emotes()


# This is so tragic omg.

role_icons_dir = os.path.join(emotes_dir, "roleIcons")
os.makedirs(role_icons_dir, exist_ok=True)
with open(os.path.join(server_dir, "info.json"), "r") as f:
    info_data = json.load(f)
os.makedirs(role_icons_dir, exist_ok=True)
print(f"Getting external emoji")

for role_data in info_data.get("team", {}).get("rolesById", {}).values():
    icon_url = role_data.get("iconUrl")
    if icon_url:
        if icon_url.startswith("http"): # Check if it looks like a valid URL
            # Append the AWS S3 parameters to the role icon URL as well
            parsed_url = urlparse(icon_url)
            query_params = urllib.parse.urlencode(AWS_PARAMS)
            modified_url = f"{icon_url}?{query_params}"

            filename = os.path.basename(urlparse(modified_url).path)
            daname = os.path.join(f"{role_data.get('id')}-", filename)
            download_path = os.path.join(role_icons_dir, filename)
            with open(download_path, "wb") as f:
                f.write(requests.get(modified_url).content)

if download_immediately:
    emojizip = os.path.join(SAVE_DIRECTORY, f"{SERVER_NAME}_emoji.zip")
    shutil.make_archive(emojizip[:-4], 'zip', emotes_dir)
    files.download(emojizip)

print("Done grabbing emoji, go to next step")

In [ ]:
# @title 1.2 Download all Media and Save HTMLs for every channel
messages_per_page = 10000 # @param {type:"integer"}
# @markdown This is the breadwinner - we will finally save and print everything, with text/stream/voice channels outputting as HTML files. It's huge and lengthy, so if you have an especially big server on top of that, you will probably want to leave and get a coffee while this runs.

# @markdown This will also split HTML files by the amount of messages, because you really don't wanna open a file over 10Mb large in your browser. 10000 should be a nice default and still might be too laggy depending on your channel, so increase it or decrease it as you need. Nothing will happen if your channel has less messages than this.

# @markdown Every other channel type will be saved only to JSONs. It isn't worth the effort to convert those. Since you will have this raw data, though, if anyone is up for it, this data can be processed later, so as long as you have the JSONs, you pretty much have everything. All media downloads will be saved as well. (I believe.)
from urllib.parse import unquote, urlparse, urlencode
verbose_mescount = True # @param {type:"boolean"}

doChats          = True # @param {type:"boolean"}
doEvents         = True # @param {type:"boolean"}
doSchedules      = True # @param {type:"boolean"}
doMedia          = True # @param {type:"boolean"}
doDocs           = True # @param {type:"boolean"}
doAnnouncements  = True # @param {type:"boolean"}
doLists          = True # @param {type:"boolean"}
doForums         = True # @param {type:"boolean"}

# Load emotes data
with open(os.path.join(server_dir, "emotes.json"), "r") as f:
    emotes_data = json.load(f)
    emotes_dict = {str(emote['id']): emote for emote in emotes_data.get('reactions', [])}
# Load server info for roles
with open(os.path.join(server_dir, "info.json"), "r") as f:
    server_info = json.load(f)
    roles_iterable = server_info['team']['rolesById'].values() if isinstance(server_info['team']['rolesById'], dict) else server_info['team']['rolesById']
    roles_data = {str(role['id']): role for role in roles_iterable}
# Load members data
with open(os.path.join(server_dir, "members.json"), "r") as f:
    members_json = json.load(f)
    members_dict = {member['id']: member for member in members_json['members']}
# Load channels data
with open(os.path.join(server_dir, "channels.json"), "r") as f:
    channels_data = json.load(f)
with open(os.path.join(server_dir, "groups.json"), "r") as f:
    groups_data = json.load(f)

def download_file(url, filename):
    # Manually construct the query string to preserve encoded characters in Signature
    query_string = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
    modified_url = f"{url}?{query_string}"

    response = requests.get(modified_url)
    if response.status_code == 200:
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "wb") as f:
            f.write(response.content)
    else:
        print(f"Error fetching data from '{modified_url}': Status code {response.status_code}")
        return None

def get_colored_name(member_id, name):
    def get_member_color(member):
        if member and 'roleIds' in member:
            for role_id in member['roleIds']:
                role = roles_data.get(str(role_id))
                if role and role['color'] != 'transparent':
                    return role['color']
        return None

    member = next((m for m in members_data['members'] if m['id'] == member_id), None)
    color = get_member_color(member)

    if color:
        return f'<span style="color: {color};">@{name}</span>'
    return f'@{name}'

def shit(url):
    match = re.match(r"^https://s3-us-west-2\.amazonaws\.com/www\.guilded\.gg/(.+)$", url)
    if match:
        return f"https://cdn.gldcdn.com/{match.group(1)}"
    return url

def dontcare(document):
    if isinstance(document, dict):
        for key, value in document.items():
            if key == "src" and isinstance(value, str):
                document[key] = shit(value)
            else:
                dontcare(value)
    elif isinstance(document, list):
        for item in value:
            dontcare(item)

def lmao(url, save_path, cookies):
    # Manually construct the query string to preserve encoded characters in Signature
    query_string = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
    modified_url = f"{url}?{query_string}"

    try:
        media_response = requests.get(modified_url, cookies=cookies)
        media_response.raise_for_status()
        os.makedirs(os.path.dirname(save_path), exist_ok=True) # Ensure directory exists before writing
        with open(save_path, "wb") as f:
            f.write(media_response.content)
        if verbose_mescount:
            print(f"Downloaded {os.path.basename(save_path)}")
    except Exception as e:
        print(f"Error downloading {os.path.basename(save_path)}: {e}")

def get_webhooks(server_dir):
    try:
        with open(os.path.join(server_dir, 'members.json'), 'r') as f:
            data = json.load(f)
            return data.get('webhooks', [])
    except FileNotFoundError:
        print("members.json not found.")
        return []
    except json.JSONDecodeError:
        print("Error decoding members.json")
        return []

webhooks_data = get_webhooks(server_dir)

def generate_html(messages, channel_data, members_data, server_dir, channel_dir):
    formatted_messages = []
    doge = {}


    with ThreadPoolExecutor() as executor:
        futures = []

        for message in messages:
            webhook_id = message.get('webhookId')
            if webhook_id:
                webhook = next((w for w in webhooks_data if w['id'] == webhook_id), None)
                created_by_id = webhook_id
            else:
                webhook = None
                created_by_id = message['createdBy']

            created_at = message.get('createdAt') or message.get('created_at')

            is_system_message = message.get('type') == 'system' or 'systemMessage' in str(message.get('content', {}))

            if webhook:
                created_by = webhook["name"]
                avatar_url = webhook.get("iconUrl") or "https://www.guilded.gg/asset/DefaultUserAvatars/profile_4.png"
                color = None
            elif is_system_message:
                member = next((m for m in members_data['members'] if m['id'] == created_by_id), None)
                created_by = member["name"] if member else "[deleted user]"
                avatar_url = "https://www.guilded.gg/asset/DefaultUserAvatars/profile_5.png"

                # Handle system message content
                content = ""
                document = message.get('content', {}).get('document', {})
                for node in document.get('nodes', []):
                    if node.get('type') == 'systemMessage':
                        data = node.get('data', {})
                        message_type = data.get('type')
                        if message_type == 'team-channel-created':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) created this chat channel."
                        elif message_type == 'channel-renamed':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) renamed this channel from '{data.get('oldName')}' to '{data.get('newName')}'."
                        elif message_type == 'streaming-screenshare-started':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) started to share their screen."
                        elif message_type == 'voice-call-started':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) started a voice call."
                        else:
                            content = f"System action performed: {message_type}"
            else:
                # Find member data based on createdBy ID
                member = next((m for m in members_data['members'] if m['id'] == created_by_id), None)
                created_by = member["name"] if member else "[deleted user]"

                # Get the base role
                base_role = next((role for role in roles_data.values() if role.get('isBase')), None)
                base_color = base_role["color"] if base_role else None

                # Get the highest priority role for the member
                highest_priority_role = None
                if member and 'roleIds' in member:
                    member_roles = [roles_data.get(str(role_id)) for role_id in member["roleIds"] if str(role_id) in roles_data]
                    member_roles.sort(key=lambda x: x["priority"], reverse=True)
                    if member_roles:
                        highest_priority_role = member_roles[0]

                # Color the created_by name based on the highest priority role or base role
                if highest_priority_role and highest_priority_role["color"] != "transparent":
                    color = highest_priority_role["color"]
                elif base_role:
                    color = base_role["color"]
                else:
                    color = None

                if color:
                    created_by = f'<span style="color: {color};">{created_by}</span>'
                else:
                    created_by = created_by


                # Handle avatar URL
                if created_by_id in members_dict and members_dict[created_by_id].get('profilePicture'):
                    avatar_url = os.path.join("..", "..", "members", members_dict[created_by_id].get('name'), os.path.basename(members_dict[created_by_id].get('profilePicture')).split('?')[0])
                else:
                    if created_by_id not in doge:
                        avatar_number = (len(doge) % 5) + 1
                        doge[created_by_id] = avatar_number
                    avatar_url = f"https://www.guilded.gg/asset/DefaultUserAvatars/profile_{doge[created_by_id]}.png"

            # Process regular message content
            content = ""
            if "content" in message and message["content"]:
                document = message["content"].get("document")
                if document:
                    nodes = document.get("nodes", [])
                    for node in nodes:
                        if node.get("type") in ("image", "video", "fileUpload"):
                            file_src = node.get("data", {}).get("src", "")
                            if file_src:
                                parsed_url = urlparse(file_src)
                                media_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

                                if node.get("type") == "fileUpload":
                                    filename = node.get("data", {}).get("name")
                                    folder = "files"
                                elif node.get("type") == "image":
                                    filename = os.path.basename(parsed_url.path)
                                    folder = "images"
                                elif node.get("type") == "video":
                                    filename = os.path.basename(parsed_url.path)
                                    folder = "videos"

                                filepath = os.path.join(channel_dir, "media", folder, filename)
                                futures.append(executor.submit(download_file, media_url, filepath))

                                if node.get("type") == "image":
                                    content += f'<img src="{os.path.join("media", "images", filename)}">'
                                elif node.get("type") == "video":
                                    content += f'<video controls><source src="{os.path.join("media", "videos", filename)}" type="video/webm">Your browser does not support the video tag.</video>'
                                elif node.get("type") == "fileUpload":
                                    file_size_bytes = node.get("data", {}).get("fileSizeBytes")
                                    file_size_kb = round(file_size_bytes / 1024, 1)
                                    content += f'''
                                        <div class="file-upload-container">
                                            <div class="file-icon icon icon-file-attach"></div>
                                            <div class="file-info">
                                                <a href="{os.path.join("media", "files", filename)}" download class="file-name">{filename}</a>
                                                <div class="file-size">{file_size_kb} kb</div>
                                            </div>
                                        </div>
                                    '''
                                content += '<br>'


                        if node.get("type") == "code-container":
                            code_lines = node.get("nodes", [])
                            code_content = ''
                            for code_line in code_lines:
                                if code_line.get("type") == "code-line":
                                    code_line_nodes = code_line.get("nodes", [])
                                    for code_line_node in code_line_nodes:
                                        leaves = code_line_node.get("leaves", [])
                                        for leaf in leaves:
                                            leaf_text = leaf.get("text", "")
                                            code_content += leaf_text + '\n'
                            language = node.get("data", {}).get("language", "")
                            if language:
                                content += f'<pre><code class="language-{language}">{code_content.strip()}</code></pre>'
                            else:
                                content += f'<pre><code>{code_content.strip()}</code></pre>'


                        elif node.get("type") == "webhookMessage":
                            embeds = node.get("data", {}).get("embeds", [])
                            for embed in embeds:
                                title = embed.get("title", "")
                                url = embed.get("url", "")
                                description = embed.get("description", "")
                                author = embed.get("author", {})
                                author_name = author.get("name", "")
                                author_url = author.get("url", "")
                                author_icon = author.get("iconUrl", "")

                                content += '<div class="webhook-embed">'
                                if title and url:
                                    content += f'<div class="embed-title"><a href="{url}">{title}</a></div>'
                                elif title:
                                    content += f'<div class="embed-title">{title}</div>'
                                if author_name:
                                    content += f'<div class="embed-author">'
                                    if author_icon:
                                        content += f'<img src="{author_icon}" class="embed-author-icon">'
                                    if author_url:
                                        content += f'<a href="{author_url}">{author_name}</a>'
                                    else:
                                        content += author_name
                                    content += '</div>'
                                if description:
                                    content += f'<div class="embed-description">{description.replace(chr(10), "<br>")}</div>'
                                content += '</div>'
                            content += '<br>'


                        elif node.get("type") == "markdown-plain-text" and node.get("data", {}).get("isEmbedMessage"):
                            sub_nodes = node.get("nodes", [])
                            for sub_node in sub_nodes:
                                leaves = sub_node.get("leaves", [])
                                for leaf in leaves:
                                    text = leaf.get("text", "")
                                    content += f'<div class="webhook-plain">{text}</div>'



                        elif node.get("type") == "paragraph":
                            sub_nodes = node.get("nodes", [])
                            for sub_node in sub_nodes:
                                if sub_node.get("type") == "reaction":
                                    reaction_data = sub_node.get("data", {})
                                    if reaction_data:
                                        reaction = reaction_data.get("reaction")
                                        if reaction:
                                            reaction_id = reaction.get("id")
                                            if reaction_id:
                                                if str(reaction_id).startswith("9") and len(str(reaction_id)) == 8:
                                                    reaction_name = reaction.get("name")
                                                    if reaction_name:
                                                        reaction_emoji = emoji.emojize(f":{reaction_name}:", language='alias')
                                                        content += reaction_emoji
                                                else:
                                                    emoji_dict = next((item for item in emotes_dict.values() if item['id'] == reaction_id), None)
                                                    if emoji_dict:
                                                        content += f'<img src="../../emotes/{emoji_dict["id"]}-{emoji_dict["name"]}.webp" alt="{emoji_dict["name"]}" title="{emoji_dict["name"]}" class="emote">'
                                                    else:
                                                        custom_reaction = reaction.get("customReaction")
                                                        if isinstance(custom_reaction, bool):
                                                            custom_reaction = None
                                                        if custom_reaction and "webp" in custom_reaction:
                                                            emote_id = custom_reaction["id"]
                                                            emote_name = custom_reaction["name"]
                                                            emote_url = custom_reaction["webp"]
                                                            emote_filename = f'{emote_id}-{emote_name}'
                                                            emotes_external_dir = os.path.join(server_dir, "emotes", "external")
                                                            if not os.path.exists(emotes_external_dir):
                                                                os.makedirs(emotes_external_dir)
                                                            emote_filepath = os.path.join(emotes_external_dir, emote_filename + ".webp")
                                                            if not os.path.exists(emote_filepath):
                                                                # Manually construct the query string for emote downloads
                                                                emote_query_string = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                                                                emote_modified_url = f"{emote_url}?{emote_query_string}"
                                                                response = requests.get(emote_modified_url)
                                                                if response.status_code == 200:
                                                                    with open(emote_filepath, "wb") as f:
                                                                        f.write(response.content)
                                                            content += f'<img src="../../emotes/external/{emote_filename}.webp" alt="{emote_name}.webp" title="{emote_name}.webp" class="emote">'





                                elif sub_node.get("type") == "mention":
                                    mention_data = sub_node.get("data", {}).get("mention", {})
                                    mentioned_id = mention_data.get("id")
                                    mentioned_name = mention_data.get("name")
                                    member = next((m for m in members_data['members'] if m['id'] == mentioned_id), None)
                                    current_name = member['name'] if member else mentioned_name
                                    display_name = f'@{mentioned_name}'
                                    content += display_name




                                #elif sub_node.get("type") == "mention":
                                #    mention_data = sub_node.get("data", {}).get("mention", {})
                                #    mentioned_id = mention_data.get("id")
                                #    mentioned_name = mention_data.get("name")
                                #    member = next((m for m in members_data['members'] if m['id'] == mentioned_id), None)
                                #    current_name = member['name'] if member else mentioned_name
                                #    color = None
                                #    if member and 'roleIds' in member:
                                #        for role_id in member['roleIds']:
                                #            role = roles_data.get(str(role_id))
                                #            if role and role['color'] != 'transparent':
                                #                color = role['color']
                                #                break
                                #    display_name = f'@{mentioned_name}'
                                #    if current_name != mentioned_name:
                                #        realname = f'Current name: @{current_name}'
                                #        if color:
                                #            content += f'<span style="color: {color};" title="{realname}">{display_name}</span>'
                                #        else:
                                #            content += f'<span title="{realname}">{display_name}</span>'
                                #    else:
                                #        if color:
                                #            content += f'<span style="color: {color};">{realname}</span>'
                                #        else:
                                #            content += display_name

                                elif sub_node.get("type") == "link":
                                    link_data = sub_node.get("data", {})
                                    href = link_data.get("href", "")
                                    if href:
                                        link_text = sub_node.get("nodes", [{}])[0].get("leaves", [{}])[0].get("text", "")
                                        content += f'<a href="{href}">{link_text}</a>'

                                elif sub_node.get("type") == "block-quote-container":
                                    block_quote_lines = sub_node.get("nodes", [])
                                    block_quote_content = ''
                                    for block_quote_line in block_quote_lines:
                                        if block_quote_line.get("type") == "block-quote-line":
                                            block_quote_line_nodes = block_quote_line.get("nodes", [])
                                            for block_quote_line_node in block_quote_line_nodes:
                                                leaves = block_quote_line_node.get("leaves", [])
                                                for leaf in leaves:
                                                    leaf_text = leaf.get("text", "")
                                                    marks = leaf.get("marks", [])
                                                    for mark in marks:
                                                        mark_type = mark.get("type")
                                                        if mark_type == "inline-code-v2":
                                                            leaf_text = f'<code>{leaf_text}</code>'
                                                        elif mark_type == "bold":
                                                            leaf_text = f'<strong>{leaf_text}</strong>'
                                                        elif mark_type == "italic":
                                                            leaf_text = f'<em>{leaf_text}</em>'
                                                        elif mark_type == "underline":
                                                            leaf_text = f'<u>{leaf_text}</u>'
                                                        elif mark_type == "strikethrough":
                                                            leaf_text = f'<del>{leaf_text}</del>'
                                                        elif mark_type == "spoiler":
                                                            leaf_text = f'<span style="background-color: #000; color: #fff;">{leaf_text}</span>'
                                                    block_quote_content += leaf_text
                                            block_quote_content += '<br>'
                                    content += f'<div style="background-color: #f0f0f0; border-left: 4px solid #F5C400; padding: 10px;">{block_quote_content}</div>'

                                else:
                                    leaves = sub_node.get("leaves", [])
                                    for leaf in leaves:
                                        leaf_text = leaf.get("text", "")
                                        marks = leaf.get("marks", [])
                                        for mark in marks:
                                            mark_type = mark.get("type")
                                            if mark_type == "inline-code-v2":
                                                if mark.get("object") == "mark":
                                                    leaf_text = sub_node.get("leaves", [{}])[0].get("text", "")
                                                    leaf_text = leaf_text.replace("<script>", "").replace("</script>", "")
                                                leaf_text = f'<code>{leaf_text}</code>'
                                            elif mark_type == "bold":
                                                leaf_text = f'<strong>{leaf_text}</strong>'
                                            elif mark_type == "italic":
                                                leaf_text = f'<em>{leaf_text}</em>'
                                            elif mark_type == "underline":
                                                leaf_text = f'<u>{leaf_text}</u>'
                                            elif mark_type == "strikethrough":
                                                leaf_text = f'<del>{leaf_text}</del>'
                                            elif mark_type == "spoiler":
                                                leaf_text = f'<span style="background-color: #000; color: #fff;">{leaf_text}</span>'
                                        content += leaf_text
                            content += '<br>'

                        elif node.get("type") == "channel":
                            chan_nodes = node.get("nodes", [])
                            #for chan_node in chan_nodes:
                            #    leaves = chan_node.get("leaves", [])
                            #    for leaf in leaves:
                            #        channel_text = leaf.get("text", "")
                            #        content += f'{channel_text}'
                            channel_data = node.get("data", {}).get("channel", {})
                            channel_name = channel_data.get("name", "")
                            if channel_name:
                                content += f'#{channel_name}'


                        elif node.get("type") == "block-quote-container":
                            block_quote_lines = node.get("nodes", [])
                            block_quote_content = ''
                            for block_quote_line in block_quote_lines:
                                if block_quote_line.get("type") == "block-quote-line":
                                    block_quote_line_nodes = block_quote_line.get("nodes", [])
                                    for block_quote_line_node in block_quote_line_nodes:
                                        leaves = block_quote_line_node.get("leaves", [])
                                        for leaf in leaves:
                                            leaf_text = leaf.get("text", "")
                                            block_quote_content += '<br>' + leaf_text.lstrip('>').strip()
                            content += f'<blockquote style="border-left: 4px solid #F5C400; padding: 4px;">{block_quote_content.strip()}</blockquote>'

                        elif node.get("type") == "unordered-list":
                            list_items = node.get("nodes", [])
                            unordered_list_content = ''
                            for list_item in list_items:
                                if list_item.get("type") == "list-item":
                                    list_item_nodes = list_item.get("nodes", [])
                                    list_item_content = ''
                                    for list_item_node in list_item_nodes:
                                        leaves = list_item_node.get("leaves", [])
                                        for leaf in leaves:
                                            leaf_text = leaf.get("text", "")
                                            list_item_content += leaf_text
                                    unordered_list_content += f'<li>{list_item_content}</li>'
                            content += f'<ul>{unordered_list_content}</ul>'
                        elif node.get("type") == "ordered-list":
                            list_items = node.get("nodes", [])
                            ordered_list_content = ''
                            for list_item in list_items:
                                if list_item.get("type") == "list-item":
                                    list_item_nodes = list_item.get("nodes", [])
                                    list_item_content = ''
                                    for list_item_node in list_item_nodes:
                                        leaves = list_item_node.get("leaves", [])
                                        for leaf in leaves:
                                            leaf_text = leaf.get("text", "")
                                            list_item_content += leaf_text
                                    ordered_list_content += f'<li>{list_item_content}</li>'
                                elif list_item.get("type") == "ordered-list":
                                    nested_ordered_list_content = ''
                                    nested_list_items = list_item.get("nodes", [])
                                    for nested_list_item in nested_list_items:
                                        if nested_list_item.get("type") == "list-item":
                                            nested_list_item_nodes = nested_list_item.get("nodes", [])
                                            nested_list_item_content = ''
                                            for nested_list_item_node in nested_list_item_nodes:
                                                nested_leaves = nested_list_item_node.get("leaves", [])
                                                for nested_leaf in nested_leaves:
                                                    nested_leaf_text = nested_leaf.get("text", "")
                                                    nested_list_item_content += nested_leaf_text
                                            nested_ordered_list_content += f'<li>{nested_list_item_content}</li>'
                                        ordered_list_content += f'<ol>{nested_ordered_list_content}</ol>'
                                content += f'<ol>{ordered_list_content}</ol>'







            formatted_message = MESSAGE_TEMPLATE.format(
                avatar_url=avatar_url,
                created_by=created_by,
                created_at=created_at,
                content=content
            )
            formatted_messages.append(formatted_message)

        # Wait for all download tasks to complete
        for future in as_completed(futures):
            future.result()

    html = HTML_TEMPLATE.format(
        title=channel_data["name"],
        topic=channel_data.get("topic", ""),
        messages='\n'.join(formatted_messages),
        CSS=CSS
    )

    return html

def get_all_src_values(obj):
    src_values = []

    if isinstance(obj, dict):
        for key, value in obj.items():
            if key == "src":
                src_values.append(value)
            else:
                src_values.extend(get_all_src_values(value))
    elif isinstance(obj, list):
        for item in obj:
            src_values.extend(get_all_src_values(item))

    return src_values

# Process each channel
for channel in channels_data["channels"]:
    # Sanitize channel name for directory creation
    channel_folder_name = sanitize_filename(channel["name"])
    channel_dir = os.path.join(server_dir, channel_folder_name)

    for group in groups_data["groups"]:
        if channel["groupId"] == group["id"]:
            # Sanitize group name for directory creation
            group_folder_name = sanitize_filename(group["name"] + " (" + group["id"] + ")")
            group_dir = os.path.join(server_dir, group_folder_name)
            channel_dir = os.path.join(group_dir, channel_folder_name)
            break

    if channel.get("contentType") in ["chat", "stream", "voice"]: # DONE. Todo maybe fix system stuff and add custom member dict but idc
        if not doChats:
            continue
        print(f"=========== FETCHING MESSAGES FROM #{channel['name']} in {group['name']}...")
        os.makedirs(channel_dir, exist_ok=True)
        with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
            json.dump(channel, f, separators=(',', ':'))
        messages = []
        beforeDate = None
        mesc = 0
        while True:
            params = {"limit": 100}
            if beforeDate:
                params["beforeDate"] = beforeDate
            response = fetch(f"channels/{channel['id']}/messages", params=params)
            messages.extend(response["messages"])
            mesc += len(response["messages"])
            #print(f"Collected {mesc} messages")
            if len(response["messages"]) < 100:
                break
            beforeDate = response["messages"][-1]["createdAt"]
        messages.reverse()
        # Save messages JSON
        with open(os.path.join(channel_dir, f"{channel['id']}_messages.json"), 'w') as f:
            json.dump(messages, f, separators=(',', ':'))
        if verbose_mescount:
            print(f"Found {len(messages)} messages")
            print(f"Saved all messages from #{channel['name']} to {channel_dir}/{channel['id']}_messages.json")
        # Generate and save HTML
        if len(messages) > messages_per_page:
            for i in range(0, len(messages), messages_per_page):
                html = generate_html(messages[i:i+messages_per_page], channel, members_json, server_dir, channel_dir)
                html_filename = f"{channel['name']} - Page {i//messages_per_page + 1}.html".replace('/', '-')
                with open(os.path.join(channel_dir, html_filename), 'w') as f:
                    f.write(html)
                if verbose_mescount:
                    print(f"Saved {channel_dir}/{html_filename}")
        else:
            html = generate_html(messages, channel, members_json, server_dir, channel_dir)
            html_filename = f"{channel['name']}.html".replace('/', '-')
            with open(os.path.join(channel_dir, html_filename), 'w') as f:
                f.write(html)
            if verbose_mescount:
                print(f"Saved {channel_dir}/{html_filename}")


    if channel.get("contentType") == "event": #DONE. Too lazy to csv
        if not doEvents:
            continue
        os.makedirs(channel_dir, exist_ok=True)
        with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
            json.dump(channel, f, separators=(',', ':'))
        cal = fetch(f"channels/{channel['id']}/events")
        with open(os.path.join(channel_dir, f"{channel['id']}_calendar.json"), 'w') as f:
            json.dump(cal, f, separators=(',', ':'))
            if verbose_mescount:
                print(f"saved {channel_dir}/{channel['id']}_calendar.json")


    if channel.get("contentType") == "scheduling": #DONE. Too lazy to csv
        if not doSchedules:
            continue
        os.makedirs(channel_dir, exist_ok=True)
        with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
            json.dump(channel, f, separators=(',', ':'))
        calurl = f"{guilded}/channels/{channel['id']}/availability"
        cal = requests.get(calurl, cookies=cookies)
        caler = cal.json()
        with open(os.path.join(channel_dir, f"{channel['id']}_schedule.json"), 'w') as f:
            json.dump(caler, f, separators=(',', ':'))
            if verbose_mescount:
                print(f"saved {channel_dir}/{channel['id']}_schedule.json")


    if channel.get("contentType") == "media": # DONE. I didn't check for same names but idm it like this personally
        if not doMedia:
            continue
        os.makedirs(channel_dir, exist_ok=True)
        with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
            json.dump(channel, f, separators=(',', ':'))
        print(f"=========== FETCHING MEDIA FROM #{channel['name']} in {group['name']}...")
        medias = []
        beforeDate = None
        while True:
            mediaurl = f"{guilded}/channels/{channel['id']}/media"
            params = {"pageSize": 40}
            if beforeDate:
                params["beforeDate"] = beforeDate
            response = requests.get(mediaurl, params=params, cookies=cookies)
            response.raise_for_status()
            data = response.json()
            for item in data:
                if "src" in item:
                    item["src"] = shit(item["src"])
            medias.extend(data)
            if len(data) < 40:
                break
            if data:
                beforeDate = data[-1]["createdAt"]
            else:
                break
        for item in medias:
            filename = os.path.basename(item["src"]).split('?')[0]
            mname = sanitize_filename(item["description"])
            mid = f"{item['id']}"
            midpath = os.path.join(channel_dir, mname)
            os.makedirs(midpath, exist_ok=True)
            filepath = os.path.join(midpath, filename)
            with open(os.path.join(midpath, f"{mid}.json"), 'w') as f:
                json.dump(item, f, separators=(',', ':'))
            lmao(item["src"], filepath, cookies)
            repliesurl = f"{guilded}/content/team_media/{mid}/replies"
            response = requests.get(repliesurl, cookies=cookies)
            response.raise_for_status()
            replies = response.json()
            if replies:
                with open(os.path.join(midpath, f"{mid}_replies.json"), 'w') as f:
                    json.dump(replies, f, separators=(',', ':'))
                replies_folder = os.path.join(midpath, "replies")
                os.makedirs(replies_folder, exist_ok=True)
                for reply in replies:
                    if "message" in reply and "document" in reply["message"]:
                        for node in reply["message"]["document"]["nodes"]:
                            if node["type"] == "image" and "data" in node and "src" in node["data"]:
                                reply_image_url = shit(node["data"]["src"])
                                reply_filename = os.path.basename(urlparse(reply_image_url).path).split('?')[0] # Apply unquote here
                                reply_filepath = os.path.join(replies_folder, reply_filename)
                                lmao(reply_image_url, reply_filepath, cookies)
        with open(os.path.join(channel_dir, f"{channel['id']}_media.json"), 'w') as f:
            json.dump(medias, f, separators=(',', ':'))


    if channel.get("contentType") == "doc": #DONE. Think we got all replies. I didn't check for same names but idm it like this personally
        if not doDocs:
            continue
        os.makedirs(channel_dir, exist_ok=True)
        with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
            json.dump(channel, f, separators=(',', ':'))
        print(f"=========== FETCHING DOCS FROM #{channel['name']} in {group['name']}...")

        docs = []
        docs_dir = os.path.join(channel_dir, "docs")
        os.makedirs(docs_dir, exist_ok=True)

        docurl = f"{guilded}/channels/{channel['id']}/docs"
        params = {"maxItems": 50}
        response = requests.get(docurl, params=params, cookies=cookies)
        response.raise_for_status()

        data = response.json()
        with open(os.path.join(channel_dir, f"{channel['id']}_docs.json"), 'w') as f:
            json.dump(data, f, separators=(',', ':'))
        for item in data:
            title = sanitize_filename(item["title"])
            item_dir = os.path.join(docs_dir, title)
            os.makedirs(item_dir, exist_ok=True)
            fulldoc = requests.get(f"{docurl}/{item['id']}", cookies=cookies)
            docdata = fulldoc.json()
            dontcare(docdata)
            with open(os.path.join(item_dir, f"{item['id']}.json"), 'w') as f:
                json.dump(docdata, f, separators=(',', ':'))

            if "content" in docdata and "document" in docdata["content"]:
                for node in docdata["content"]["document"]["nodes"]:
                    if node["type"] == "image" and "data" in node:
                        upload = node["data"]
                        if "name" in upload and "src" in upload:
                            original_filename = upload["name"]  # This is the name of the file
                            download_url = upload["src"]  # This is the URL to download the file from
                            parsed_url = urllib.parse.urlparse(download_url)
                            url_path = parsed_url.path
                            _, url_extension = os.path.splitext(url_path)
                            filename = f"{original_filename}{url_extension}"
                            saveto = os.path.join(docs_dir, title, filename)
                            try:
                                # Manually construct the query string for doc image downloads
                                download_parsed_url = urlparse(download_url)
                                download_query_params = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                                download_modified_url = f"{download_url}?{download_query_params}"

                                response = requests.get(download_modified_url, cookies=cookies)
                                response.raise_for_status()
                                with open(saveto, "wb") as f:
                                    f.write(response.content)
                                if verbose_mescount:
                                    print(f"Downloaded {filename} from {download_modified_url}")
                            except requests.RequestException as e:
                                print(f"Error downloading {filename} from {download_modified_url}: {e}")


    if channel.get("contentType") == "announcement": #DONE, think we got replies too
        if not doAnnouncements:
            continue
        os.makedirs(channel_dir, exist_ok=True)
        with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
            json.dump(channel, f, separators=(',', ':'))
        print(f"=========== FETCHING ANNOUNCEMENTS FROM #{channel['name']} in {group['name']}...")
        annurl = f"{guilded}/channels/{channel['id']}/announcements"
        anns = []
        params = {"maxItems": 3}
        beforeDate = None
        while True:
            if beforeDate:
                params["beforeDate"] = beforeDate
            response = requests.get(annurl, params=params, cookies=cookies)
            response.raise_for_status()
            data = unshid_cdn(response.json())
            anns.extend(data["announcements"])
            if len(data["announcements"]) < 3:
                break
            if data["announcements"]:
                beforeDate = data["announcements"][-1]["createdAt"]
            else:
                break
        with open(os.path.join(channel_dir, f"{channel['id']}_announcements.json"), 'w') as f:
            json.dump(anns, f, separators=(',', ':'))
        for ann in anns:
            annid = ann["id"]
            title = sanitize_filename(ann["title"]) # Sanitize title for directory creation
            annpath = os.path.join(channel_dir, title)
            os.makedirs(annpath, exist_ok=True)
            with open(os.path.join(annpath, f"{annid}.json"), 'w') as f:
                json.dump(ann, f, separators=(',', ':'))

            links = []
            def find_links(data):
                if isinstance(data, dict):
                    if "src" in data:
                        links.append(data["src"])
                    else:
                        for value in data.values():
                            find_links(value)
                elif isinstance(data, list):
                    for item in data:
                        find_links(item)
            find_links(ann["content"]["document"]["nodes"])
            mediadir = os.path.join(annpath, "media")
            for link in links:
                url = link
                os.makedirs(mediadir, exist_ok=True)
                parsed_url = urllib.parse.urlparse(url)
                filename = os.path.basename(parsed_url.path)
                filepath = os.path.join(mediadir, filename)

                # Manually construct the query string for announcement media downloads
                download_parsed_url = urlparse(url)
                download_query_params = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                download_modified_url = f"{url}?{download_query_params}"

                response = requests.get(download_modified_url)
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                    if verbose_mescount:
                        print(f"Downloaded {filepath}")

            annrep = f"{guilded}/content/announcement/{annid}/replies"
            rep_response = requests.get(annrep, cookies=cookies).json()

            with open(os.path.join(annpath, f"{annid}_replies.json"), 'w') as f:
                json.dump(rep_response, f, separators=(',', ':'))

            links = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(rep_response))
            links = [re.sub(r'^https://s3-us-west-2\.amazonaws\.com/www\.guilded\.gg/', 'https://cdn.gldcdn.com/', url) for url in links]
            for url in links:
                os.makedirs(mediadir, exist_ok=True)

                # Manually construct the query string for announcement reply downloads
                download_parsed_url = urlparse(url)
                download_query_params = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                download_modified_url = f"{url}?{download_query_params}"

                response = requests.get(download_modified_url, stream=True)
                if response.status_code == 200:
                    filename = os.path.basename(urlparse(response.url).path).split('?')[0] # Apply unquote here
                    filepath = os.path.join(mediadir, filename)
                    with open(filepath, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=1024):
                            if chunk:
                                f.write(chunk)
                        if verbose_mescount:
                            print (f"Downloaded {filepath}")


    if channel.get("contentType") == "list": # DONE, no need to process.
        if not doLists:
            continue
        os.makedirs(channel_dir, exist_ok=True)
        with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
            json.dump(channel, f, separators=(',', ':'))
        print(f"=========== FETCHING TODO FROM #{channel['name']} in {group['name']}...")
        os.makedirs(channel_dir, exist_ok=True)
        todourl = f"{guilded}/channels/{channel['id']}/listitems"
        response = requests.get(todourl, cookies=cookies)
        response.raise_for_status()
        todo = response.json()
        with open(os.path.join(channel_dir, f"{channel['id']}_todo.json"), 'w') as f:
            json.dump(todo, f, separators=(',', ':'))


    if channel.get("contentType") == "forum": # DOOOOONE. May we all love one day.
        if not doForums:
            continue
        print(f"=========== FETCHING THREADS FROM #{channel['name']}...")
        os.makedirs(channel_dir, exist_ok=True)
        with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
            json.dump(channel, f, separators=(',', ':'))
        all_threads = []
        page = 1

        while True:
            forumurl = f"{guilded}/channels/{channel['id']}/forums"
            params = {"maxItems": 30, "page": page}

            response = requests.get(forumurl, params=params, cookies=cookies)
            response.raise_for_status()
            data = unshid_cdn(response.json())

            threads = data.get("threads", [])
            if not threads:
                break
            all_threads.extend(threads)
            page += 1

            with open(os.path.join(channel_dir, f"{channel['id']}_threads.json"), 'w') as f:
                json.dump(threads, f, separators=(',', ':'))



            for thread in data.get("threads", []):
                thread_id = thread["id"]
                thread_title = sanitize_filename(thread["title"]) # Sanitize title for directory creation
                thread_dir = os.path.join(channel_dir, thread_title)
                os.makedirs(thread_dir, exist_ok=True)
                if verbose_mescount:
                    print(f"Fetching {thread_id}")
                thread_url = f"{guilded}/channels/{channel['id']}/forums/{thread_id}"
                thread_response = unshid_cdn(requests.get(thread_url, cookies=cookies).json())
                os.makedirs(os.path.join(channel_dir, "threads"), exist_ok=True)
                with open(os.path.join(thread_dir, f"{thread_id}_thread.json"), 'w') as f:
                    json.dump(thread_response, f, separators=(',', ':'))
                media_dir = os.path.join(thread_dir, "media")


                afterDate = None
                postc = 0
                posts = []
                while True:
                    params = {"maxItems": 50}
                    if afterDate:
                        params["afterDate"] = afterDate

                    response = fetch(f"channels/{channel['id']}/forums/{thread_id}/replies", params=params)
                    posts.extend(response["threadReplies"])

                    # Check if fewer than 50 replies were returned, indicating the last page
                    if len(response["threadReplies"]) < 50:
                        break

                    afterDate = response["threadReplies"][-1]["createdAt"]

                # Write the complete posts to a JSON file
                with open(os.path.join(thread_dir, f"{thread_id}_replies.json"), 'w') as f:
                    json.dump(posts, f, separators=(',', ':'))


                # This is terrible.
                os.makedirs(media_dir, exist_ok=True)

                for post in posts:
                    if "message" in post and "document" in post["message"]:
                        for node in post["message"]["document"]["nodes"]:
                            if node["object"] == "block":
                                if node["type"] == "fileUpload" and "data" in node:
                                    file_url = node["data"]["src"]
                                    realname = os.path.splitext(node["data"]["name"])[0]
                                    up_name = unquote(os.path.basename(urlparse(file_url).path)).replace("-Full", "")
                                    file_name = f"{realname}_{up_name}"
                                    file_path = os.path.join(media_dir, file_name)
                                elif node["type"] in ["image", "video"] and "data" in node:
                                    file_url = node["data"]["src"]
                                    parsed_url = urlparse(file_url)
                                    url_path = parsed_url.path
                                    _, url_extension = os.path.splitext(url_path)
                                    file_name = f"{unquote(os.path.basename(url_path))}"
                                    file_path = os.path.join(media_dir, file_name)
                                else:
                                    continue

                                try:
                                    # Manually construct the query string for forum reply downloads
                                    download_parsed_url = urlparse(file_url)
                                    download_query_params = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                                    download_modified_url = f"{file_url}?{download_query_params}"

                                    media_response = requests.get(download_modified_url)
                                    media_response.raise_for_status()
                                    with open(file_path, 'wb') as f:
                                        f.write(media_response.content)
                                    if verbose_mescount:
                                        print(f"Downloaded {file_name} from {download_modified_url}")
                                except requests.RequestException as e:
                                    print(f"Error downloading {file_name} from {download_modified_url}: {e}")


                for node in thread_response["thread"]["message"]["document"]["nodes"]:
                    if node["object"] == "block":
                        if node["type"] == "fileUpload" and "data" in node:
                            file_url = node["data"]["src"]
                            realname = os.path.splitext(node["data"]["name"])[0]
                            up_name = unquote(os.path.basename(urlparse(file_url).path)).replace("-Full", "")
                            file_name = f"{realname}_{up_name}"
                            file_path = os.path.join(media_dir, file_name)
                        elif node["type"] in ["image", "video"] and "data" in node:
                            file_url = node["data"]["src"]
                            parsed_url = urlparse(file_url)
                            url_path = parsed_url.path
                            _, url_extension = os.path.splitext(url_path)
                            file_name = f"{unquote(os.path.basename(url_path))}"
                            file_path = os.path.join(media_dir, file_name)
                        else:
                            continue

                        try:
                            # Manually construct the query string for forum thread downloads
                            download_parsed_url = urlparse(file_url)
                            download_query_params = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                            download_modified_url = f"{file_url}?{download_query_params}"

                            media_response = requests.get(download_modified_url)
                            media_response.raise_for_status()
                            with open(file_path, 'wb') as f:
                                f.write(media_response.content)
                            if verbose_mescount:
                                print(f"Downloaded {file_name} from {download_modified_url}")
                        except requests.RequestException as e:
                            print(f"Error downloading {file_name} from {download_modified_url}: {e}")


for group in groups_data["groups"]:
    group_dir = os.path.join(server_dir, sanitize_filename(group["name"] + " (" + group["id"] + ")")) # Sanitize group name for directory creation
    if group.get("avatar"):
        parsed_url = urlparse(group["avatar"]) # Parse the URL
        pic_filename = os.path.basename(parsed_url.path) # Extract filename from parsed URL's path
        if not os.path.exists(os.path.join(group_dir, pic_filename)):
            os.makedirs(group_dir, exist_ok=True)
        pic_filepath = os.path.join(group_dir, pic_filename)
        response = requests.get(group["avatar"]) # Pass original URL string to requests.get
        if response.status_code == 200:
            with open(pic_filepath, "wb") as f:
                f.write(response.content)

print("Finished processing all channels.")

In [ ]:
# @title 1.3 Download the Whole Server
# @markdown You should run this anyway because this will clean lots of redundant empty files and folders.

# @markdown If your output archive seems way too big (more than 2GB), DDL will be very unreliable. You should use GDrive instead or find a way to get Colab to send it to you.

for dirpath, dirnames, filenames in os.walk(server_dir):
  for filename in filenames:
    filepath = os.path.join(dirpath, filename)
    if os.path.isfile(filepath) and os.stat(filepath).st_size == 2:
      with open(filepath, 'r') as f:
        content = f.read()
        if content == '[]':
          os.remove(filepath)
          #print(f"cleaned {filepath}")
  for dirname in dirnames:
    dirpath_to_check = os.path.join(dirpath, dirname)
    if not os.listdir(dirpath_to_check):
      os.rmdir(dirpath_to_check)
      #print(f"cleaned {dirpath_to_check}")

import shutil
import subprocess
from google.colab import drive
from google.colab import files
save_type = "7z" # @param ["7z", "targz", "zip"]
saving_to = "DirectDownLoad" # @param ["DirectDownLoad", "GoogleDrive"]
todl = SAVE_DIRECTORY + f"{SERVER_NAME}"
if save_type == "7z":
    outfile = f"{SAVE_DIRECTORY}{SERVER_NAME}.7z"
    !7z a -mx0 "{outfile}" "{todl}"
if save_type == "targz":
    outfile = f"{SAVE_DIRECTORY}{SERVER_NAME}.tar.gz"
    !tar -czvf "{outfile}" "{todl}"
if save_type == "zip":
    outfile = f"{SAVE_DIRECTORY}{SERVER_NAME}.zip"
    !zip -r "{outfile}" "{todl}"

if saving_to == "DirectDownLoad":
    files.download(outfile)
if saving_to == "GoogleDrive":
    drive.mount('/content/drive')
    !cp "{outfile}" /content/drive/MyDrive

# Direct Messages

In [ ]:
# @title 2.0 Export all DMs and Media
messages_per_page = 15000 # @param {type:"integer"}
# @markdown Same deal as above, but you can probably increase the message count way more as DMs aren't as scary as servers. This also works on Groups.
from urllib.parse import urlparse, unquote, urlencode, parse_qsl
# @markdown Make sure to at least tick the HTML File Setup cell if you skipped doing any server stuff.
verbose_dmfetch = True # @param {type:"boolean"}

def download_file(url, filename):
    # Check if the URL already has query parameters from AWS
    parsed_url = urlparse(url)
    query_dict = dict(parse_qsl(parsed_url.query))

    if not all(key in query_dict for key in AWS_PARAMS):
        # Manually construct the query string to preserve encoded characters in Signature
        query_string = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
        # Append the query string to the base URL (without existing query)
        modified_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}?{query_string}"
    else:
        modified_url = url # Use the original URL if AWS params are already present


    try:
        response = requests.get(modified_url, timeout=45)
        if response.status_code == 200:
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            with open(filename, "wb") as f:
                f.write(response.content)
        else:
            print(f"Error downloading {modified_url}: Status code {response.status_code}")
    except requests.exceptions.Timeout:
        print(f"Timeout occurred while downloading {modified_url}. Skipping...")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {modified_url}: {e}")


# ... rest of the code remains the same as servers ...
def generate_html(messages, dm_data, members_data, dms_dir):
    formatted_messages = []
    doge = {}

    with ThreadPoolExecutor() as executor:
        futures = []

        for message in messages:
            created_by_id = message['createdBy']
            created_at = message.get('createdAt') or message.get('created_at')

            # Determine if it's a system message
            is_system_message = message.get('type') == 'system' or 'systemMessage' in str(message.get('content', {}))

            if is_system_message:
                member = next((m for m in dm_data['users'] if m['id'] == created_by_id), None)
                created_by = member["name"] if member else "[deleted user]"
                avatar_url = ""

                # Handle system message content
                content = ""
                document = message.get('content', {}).get('document', {})
                for node in document.get('nodes', []):
                    if node.get('type') == 'systemMessage':
                        data = node.get('data', {})
                        message_type = data.get('type')
                        if message_type == 'team-channel-created':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) created this chat channel."
                        elif message_type == 'channel-renamed':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) renamed this channel from '{data.get('oldName')}' to '{data.get('newName')}'."
                        elif message_type == 'streaming-screenshare-started':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) started to share their screen."
                        elif message_type == 'voice-call-started':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) started a voice call."
                        elif message_type == 'webhookMessage':
                            embeds = node.get('data', {}).get('embeds', [])
                            content = json.dumps(data, indent=2)
                        else:
                            content = f"System action performed: {message_type}"


            else:
                # Find member data based on createdBy ID
                member = next((m for m in dm_data['users'] if m['id'] == created_by_id), None)
                created_by = member["name"] if member else "[deleted user]"
                #print("Member data:", member)  # Print the member data for inspection

                if member and member.get('profilePicture'):
                    profile_picture_url = member['profilePicture']
                    filename = f"{created_by_id}{os.path.splitext(os.path.basename(unquote(urlparse(profile_picture_url).path)))[1]}"
                    filepath = os.path.join(dm_dir, "media", filename)
                    os.makedirs(os.path.dirname(filepath), exist_ok=True)
                    with open(filepath, "wb") as f:
                        # Manually construct the query string for profile picture downloads
                        profile_parsed_url = urlparse(profile_picture_url)
                        profile_query_params = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                        profile_modified_url = f"{profile_picture_url}?{profile_query_params}"
                        f.write(requests.get(profile_modified_url).content)
                    avatar_url = os.path.join("media", filename)
                else:
                    avatar_url = f"https://www.guilded.gg/asset/DefaultUserAvatars/profile_{doge.setdefault(created_by_id, (len(doge) % 5) + 1)}.png"


                # Process regular message content
                content = ""
                if "content" in message and message["content"]:
                    document = message["content"].get("document")
                    if document:
                        nodes = document.get("nodes", [])
                        for node in nodes:
                            if node.get("type") in ("image", "video", "fileUpload"):
                                file_src = node.get("data", {}).get("src", "")
                                if file_src:
                                    parsed_url = urlparse(file_src)
                                    media_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

                                    if node.get("type") == "fileUpload":
                                        file_src = node.get("data", {}).get("src", "")
                                        if file_src:
                                            parsed_url = urlparse(file_src)
                                            media_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

                                            #filename = f"{node.get('data', {}).get('name')}_{message['id']}"
                                            filename = f"{node.get('data', {}).get('name')}_{message['id']}{os.path.splitext(os.path.basename(unquote(urlparse(file_src).path)))[1]}"
                                            folder = "files"
                                            filepath = os.path.join(dm_dir, "media", folder, filename)
                                            os.makedirs(os.path.dirname(filepath), exist_ok=True)

                                            # Manually construct the query string for DM file uploads
                                            download_parsed_url = urlparse(media_url)
                                            download_query_params = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                                            download_modified_url = f"{media_url}?{download_query_params}"

                                            futures.append(executor.submit(download_file, download_modified_url, filepath))



                                    elif node.get("type") == "image":
                                        filename = os.path.basename(parsed_url.path)
                                        folder = "images"
                                    elif node.get("type") == "video":
                                        filename = os.path.basename(parsed_url.path)
                                        folder = "videos"

                                    filepath = os.path.join(dm_dir, "media", folder, filename)

                                    # Manually construct the query string for DM media
                                    download_parsed_url = urlparse(media_url)
                                    download_query_params = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                                    download_modified_url = f"{media_url}?{download_query_params}"

                                    futures.append(executor.submit(download_file, download_modified_url, filepath))
                                    if verbose_dmfetch:
                                        print(f"{filename}")

                                    if node.get("type") == "image":
                                        content += f'<img src="{os.path.join("media", "images", filename)}">'
                                    elif node.get("type") == "video":
                                        content += f'<video controls><source src="{os.path.join("media", "videos", filename)}" type="video/webm">Your browser does not support the video tag.</video>'
                                    elif node.get("type") == "fileUpload":
                                        file_size_bytes = node.get("data", {}).get("fileSizeBytes")
                                        file_size_kb = round(file_size_bytes / 1024, 1)
                                        content += f'''
                                            <div class="file-upload-container">
                                                <div class="file-icon icon icon-file-attach"></div>
                                                <div class="file-info">
                                                    <a href="{os.path.join("media", "files", filename)}" download class="file-name">{node.get('data', {}).get('name')}</a>
                                                    <div class="file-size">{file_size_kb} kb</div>
                                                </div>
                                            </div>
                                        '''


                            if node.get("type") == "code-container":
                                code_lines = node.get("nodes", [])
                                code_content = ''
                                for code_line in code_lines:
                                    if code_line.get("type") == "code-line":
                                        code_line_nodes = code_line.get("nodes", [])
                                        for code_line_node in code_line_nodes:
                                            leaves = code_line_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                code_content += leaf_text + '\n'
                                language = node.get("data", {}).get("language", "")
                                if language:
                                    content += f'<pre><code class="language-{language}">{code_content.strip()}</code></pre>'
                                else:
                                    content += f'<pre><code>{code_content.strip()}</code></pre>'


                            elif node.get("type") == "paragraph":
                                sub_nodes = node.get("nodes", [])
                                for sub_node in sub_nodes:
                                    if sub_node.get("type") == "reaction":
                                        reaction_data = sub_node.get("data", {})
                                        if reaction_data and (reaction := reaction_data.get("reaction")) and (reaction_id := reaction.get("id")):
                                            if str(reaction_id).startswith("9") and len(str(reaction_id)) == 8:
                                                content += emoji.emojize(f":{reaction.get('name')}:", language='alias') if reaction.get('name') else ''
                                            else:

                                                custom_reaction = reaction.get("customReaction") or {}
                                                if isinstance(custom_reaction, bool):
                                                    custom_reaction = {}
                                                emote_url = custom_reaction.get("webp")
                                                emote_filename = custom_reaction.get("name")
                                                emote_id = custom_reaction.get("id")
                                                if emote_url and emote_filename:
                                                    emote_filepath = os.path.join(dms_dir, "media", "emotes", f"{emote_id}-{emote_filename}.webp")
                                                    if not os.path.exists(emote_filepath):
                                                        try:
                                                            # Manually construct the query string for DM emotes
                                                            emote_parsed_url = urlparse(emote_url)
                                                            emote_query_params = "&".join([f"{key}={value}" for key, value in AWS_PARAMS.items()])
                                                            emote_modified_url = f"{emote_url}?{emote_query_params}"

                                                            response = requests.get(emote_modified_url)
                                                            if response.status_code == 200:
                                                                os.makedirs(os.path.dirname(emote_filepath), exist_ok=True)
                                                                with open(emote_filepath, "wb") as f:
                                                                    f.write(response.content)
                                                            else:
                                                                print(f"Failed to download custom reaction: {emote_filename} (Status code: {response.status_code})")
                                                        except requests.exceptions.RequestException as e:
                                                            print(f"Error downloading custom reaction: {emote_filename}")
                                                            print(e)
                                                            pass
                                                    content += f'<img src="{os.path.join("media", "emotes", f"{emote_id}-{emote_filename}.webp")} alt="{emote_filename}" title="{emote_filename}" class="emote">'


                                    elif sub_node.get("type") == "mention":
                                        mention_data = sub_node.get("data", {}).get("mention", {})
                                        mentioned_id = mention_data.get("id")
                                        mentioned_name = mention_data.get("name")
                                        display_name = f'@{mentioned_name}'
                                        content += display_name

                                    elif sub_node.get("type") == "link":
                                        link_data = sub_node.get("data", {})
                                        href = link_data.get("href", "")
                                        if href:
                                            link_text = sub_node.get("nodes", [{}])[0].get("leaves", [{}])[0].get("text", "")
                                            content += f'<a href="{href}">{link_text}</a>'

                                    elif sub_node.get("type") == "block-quote-container":
                                        block_quote_lines = sub_node.get("nodes", [])
                                        block_quote_content = ''
                                        for block_quote_line in block_quote_lines:
                                            if block_quote_line.get("type") == "block-quote-line":
                                                block_quote_line_nodes = block_quote_line.get("nodes", [])
                                                for block_quote_line_node in block_quote_line_nodes:
                                                    leaves = block_quote_line_node.get("leaves", [])
                                                    for leaf in leaves:
                                                        leaf_text = leaf.get("text", "")
                                                        marks = leaf.get("marks", [])
                                                        for mark in marks:
                                                            mark_type = mark.get("type")
                                                            if mark_type == "inline-code-v2":
                                                                leaf_text = f'<code>{leaf_text}</code>'
                                                            elif mark_type == "bold":
                                                                leaf_text = f'<strong>{leaf_text}</strong>'
                                                            elif mark_type == "italic":
                                                                leaf_text = f'<em>{leaf_text}</em>'
                                                            elif mark_type == "underline":
                                                                leaf_text = f'<u>{leaf_text}</u>'
                                                            elif mark_type == "strikethrough":
                                                                leaf_text = f'<del>{leaf_text}</del>'
                                                            elif mark_type == "spoiler":
                                                                leaf_text = f'<span style="background-color: #000; color: #fff;">{leaf_text}</span>'
                                                        block_quote_content += leaf_text
                                                block_quote_content += '<br>'
                                        content += f'<div style="background-color: #f0f0f0; border-left: 4px solid #F5C400; padding: 10px;">{block_quote_content}</div>'


                                    else:
                                        leaves = sub_node.get("leaves", [])
                                        for leaf in leaves:
                                            leaf_text = leaf.get("text", "")
                                            marks = leaf.get("marks", [])
                                            for mark in marks:
                                                mark_type = mark.get("type")
                                                if mark_type == "inline-code-v2":
                                                    leaf_text = f'<code>{leaf_text}</code>'
                                                elif mark_type == "bold":
                                                    leaf_text = f'<strong>{leaf_text}</strong>'
                                                elif mark_type == "italic":
                                                    leaf_text = f'<em>{leaf_text}</em>'
                                                elif mark_type == "underline":
                                                    leaf_text = f'<u>{leaf_text}</u>'
                                                elif mark_type == "strikethrough":
                                                    leaf_text = f'<del>{leaf_text}</del>'
                                            content += leaf_text
                                content += '<br>'

                            elif node.get("type") == "block-quote-container":
                                block_quote_lines = node.get("nodes", [])
                                block_quote_content = ''
                                for block_quote_line in block_quote_lines:
                                    if block_quote_line.get("type") == "block-quote-line":
                                        block_quote_line_nodes = block_quote_line.get("nodes", [])
                                        for block_quote_line_node in block_quote_line_nodes:
                                            leaves = block_quote_line_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                block_quote_content += '<br>' + leaf_text.lstrip('>').strip()
                                content += f'<blockquote style="border-left: 44px solid #F5C400; padding: 4px;">{block_quote_content.strip()}</blockquote>'

                            elif node.get("type") == "unordered-list":
                                list_items = node.get("nodes", [])
                                unordered_list_content = ''
                                for list_item in list_items:
                                    if list_item.get("type") == "list-item":
                                        list_item_nodes = list_item.get("nodes", [])
                                        list_item_content = ''
                                        for list_item_node in list_item_nodes:
                                            leaves = list_item_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                list_item_content += leaf_text
                                        unordered_list_content += f'<li>{list_item_content}</li>'
                                content += f'<ul>{unordered_list_content}</ul>'
                            elif node.get("type") == "ordered-list":
                                list_items = node.get("nodes", [])
                                ordered_list_content = ''
                                for list_item in list_items:
                                    if list_item.get("type") == "list-item":
                                        list_item_nodes = list_item.get("nodes", [])
                                        list_item_content = ''
                                        for list_item_node in list_item_nodes:
                                            leaves = list_item_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                list_item_content += leaf_text
                                        ordered_list_content += f'<li>{list_item_content}</li>'
                                    elif list_item.get("type") == "ordered-list":
                                        nested_ordered_list_content = ''
                                        nested_list_items = list_item.get("nodes", [])
                                        for nested_list_item in nested_list_items:
                                            if nested_list_item.get("type") == "list-item":
                                                nested_list_item_nodes = nested_list_item.get("nodes", [])
                                                nested_list_item_content = ''
                                                for nested_list_item_node in nested_list_item_nodes:
                                                    nested_leaves = nested_list_item_node.get("leaves", [])
                                                    for nested_leaf in nested_leaves:
                                                        nested_leaf_text = nested_leaf.get("text", "")
                                                        nested_list_item_content += nested_leaf_text
                                                nested_ordered_list_content += f'<li>{nested_list_item_content}</li>'
                                        ordered_list_content += f'<ol>{nested_ordered_list_content}</ol>'
                                content += f'<ol>{ordered_list_content}</ol>'


            formatted_message = MESSAGE_TEMPLATE.format(
                avatar_url=avatar_url,
                created_by=created_by,
                created_at=created_at,
                content=content
            )
            formatted_messages.append(formatted_message)

        # Wait for all download tasks to complete
        for future in as_completed(futures):
            future.result()

    html = HTML_TEMPLATE.format(
        title=dm_data["name"],
        topic=", ".join(member["name"] for member in dm_data["users"]),
        messages='\n'.join(formatted_messages),
        CSS=CSS
    )

    return html



myself_data = fetch_myself()
with open(os.path.join(SAVE_DIRECTORY, "myself.json"), "w") as f:
    f.write(json.dumps(myself_data, separators=(',', ':')))
    print(f"Saved {SAVE_DIRECTORY}myself.json")
USER_ID = myself_data["user"]["id"]
dms_data = fetch_dms()
with open(os.path.join(SAVE_DIRECTORY, "dms.json"), "w") as f:
    f.write(json.dumps(dms_data, separators=(',', ':')))
    print(f"Saved {SAVE_DIRECTORY}dms.json")


for channel in dms_data['channels']:
    channel_id = channel['id']
    if verbose_dmfetch:
        print(f"found {channel_id}")
    # Use the channel data directly from dms_data instead of calling fetch_channel
    dm_data = channel
    partner = next((user for user in dm_data['users'] if user['id'] != USER_ID), None)
    if dm_data.get("dmType") == "Group":
        dm_foldername = f"Group {channel_id}"
    elif partner:
        dm_foldername = f"{partner['name']} ({partner['id']})"
    else:
        dm_foldername = f"[deleted user] ({channel_id})"
    dm_dir = os.path.join(SAVE_DIRECTORY, "Direct Messages", sanitize_filename(dm_foldername))

    # Check if the directory already exists
    if os.path.exists(dm_dir):
        print(f"Skipping existing directory: {dm_dir}")
        continue  # Skip to the next channel

    os.makedirs(dm_dir, exist_ok=True)
    with open(os.path.join(dm_dir, f"{channel_id}_info.json"), 'w') as f:
        json.dump(dm_data, f, separators=(',', ':'))
        if verbose_dmfetch:
            print(f"Saving channel {channel_id} for user {partner['name'] if partner else 'Unknown'}")
    members_dict = {}
    for member in dm_data['users']:
        members_dict[member['name']] = member

    url = f"https://www.guilded.gg/api/channels/{channel_id}/messages"
    beforeDate = None # Reset beforeDate for each channel
    messages = [] # Reset messages for each channel
    mesc = 0 # Reset mesc for each channel
    while True:
        params = {"maxItems": 100, "beforeDate": beforeDate}
        response = requests.get(url, params=params, cookies=cookies)
        dm_mes = unshid_cdn(response.json())
        #response = fetch(f"api/channels/{channel_id}/messages", params={"maxItems": 100, "beforeDate": beforeDate})
        messages.extend(dm_mes["messages"])
        mesc += len(dm_mes["messages"])
        if verbose_dmfetch:
            print(f"Collected {mesc} messages")
        if len(dm_mes["messages"]) < 100:
            break
        beforeDate = dm_mes["messages"][-1]["createdAt"]
    messages.reverse()
    # Save messages JSON
    with open(os.path.join(dm_dir, f"{channel_id}_messages.json"), 'w') as f:
        json.dump(messages, f, separators=(',', ':'))
    if verbose_dmfetch:
        print(f"Found {len(messages)} messages")

    # Generate and save HTML
    if len(messages) > messages_per_page:
        for i in range(0, len(messages), messages_per_page):
            html = generate_html(messages[i:i+messages_per_page], dm_data, members_dict, dm_dir)
            html_filename = f"DMs with {partner['name']} - Page {i//messages_per_page + 1}.html".replace('/', '-')
            if dm_data.get("dmType") == "Group":
                html_filename = f"Group {channel_id} - Page {i//messages_per_page + 1}.html".replace('/', '-')
            with open(os.path.join(dm_dir, html_filename), 'w') as f:
                f.write(html)
            if verbose_dmfetch:
                print(f"Saved {dm_dir}/{html_filename}")
    else:
        html = generate_html(messages, dm_data, members_dict, dm_dir)
        html_filename = f"DMs with {partner['name']}.html".replace('/', '-')
        if dm_data.get("dmType") == "Group":
            html_filename = f"Group {channel_id}.html".replace('/', '-')
        with open(os.path.join(dm_dir, html_filename), 'w') as f:
            f.write(html)

    print(f"Saved HTML to {dm_dir}/{html_filename}")

print("Finished processing all DMs.")

In [ ]:
# @title (Optional) Get the size of the directory SAVE_DIRECTORY + "Direct Messages"

import os

todl = SAVE_DIRECTORY + "Direct Messages"

size = 0
for root, dirs, files in os.walk(todl):
    for file in files:
        size += os.path.getsize(os.path.join(root, file))

mb = size / (1024 * 1024)
gb = size / (1024 * 1024 * 1024)

print(f"Size of directory: {mb:.2f} MB / {gb:.2f} GB")


In [ ]:
# @title 2.1 Download and Save DMs
save_type = "7z" # @param ["7z", "targz", "zip"]
saving_to = "DirectDownLoad" # @param ["DirectDownLoad", "GoogleDrive"]
from google.colab import drive
from google.colab import files

todl = SAVE_DIRECTORY + "Direct Messages"
if save_type == "7z":
    outfile = f"{SAVE_DIRECTORY}Direct_Messages.7z"
    !7z a -mx {outfile} "{todl}" -mx0
if save_type == "targz":
    outfile = f"{SAVE_DIRECTORY}Direct_Messages.tar.gz"
    !tar -czvf {outfile} "{todl}"
if save_type == "zip":
    outfile = f"{SAVE_DIRECTORY}Direct_Messages.zip"
    !zip -r {outfile} "{todl}"

if saving_to == "DirectDownLoad":
    files.download(outfile)
if saving_to == "GoogleDrive":
    drive.mount('/content/drive')
    !cp "{outfile}" /content/drive/MyDrive

#if save_to_gdrive:
    #shutil.copy(SERVER_NAME + '.zip', '/content/drive/MyDrive/' + SERVER_NAME + '.zip')
    #print("Copied zip to Google Drive.")


In [ ]:
# @title (Very Nice to Have) Profile Info Nabber (Self and Friends)
me = fetch_myself()
friends = me.get("friends")
friend_ids = [friend["friendUserId"] for friend in friends]
myname = me.get('user', {}).get('name')
def fetch_userp(user_id):
    data = fetch(f"users/{user_id}")
    return data["user"]

profiledir = os.path.join(SAVE_DIRECTORY, "Profiles")
data = fetch_userp(me.get('user', {}).get('id'))
os.makedirs(profiledir, exist_ok=True)
with open(os.path.join(profiledir, f"ProfileMyself-{myname}.json"), "w") as f:
        f.write(json.dumps(data, separators=(',', ':')))
        print(f"Saved {profiledir}/ProfileMyself-{myname}.json")
for member in dm_data["users"]:
    data = fetch_user(member["id"])
    with open(os.path.join(profiledir, f"{member['id']}-{member['name']}.json"), "w") as f:
        f.write(json.dumps(data, separators=(',', ':')))
        print(f"Saved {profiledir}/{member['id']}-{member['name']}.json")
for friend_id in friend_ids:
    data = fetch_user(friend_id)
    with open(os.path.join(profiledir, f"{friend_id}-{data['name']}.json"), "w") as f:
        f.write(json.dumps(data, separators=(',', ':')))
        print(f"Saved {profiledir}/{friend_id}-{data['name']}.json")
for member in members_dict:
    data = fetch_user(members_dict[member]["id"])
    with open(os.path.join(profiledir, f"{members_dict[member]['id']}-{members_dict[member]['name']}.json"), "w") as f:
        f.write(json.dumps(data, separators=(',', ':')))
        print(f"Saved {profiledir}/{members_dict[member]['id']}-{members_dict[member]['name']}")

# zip and download profiledir
outfile = f"{SAVE_DIRECTORY}Profiles.7z"
!7z a {outfile} {profiledir}
files.download(outfile)


In [ ]:
# @title (Debug) Preview HTML (wont show local files)
# @markdown You can test how messages look. But you have to put the link manually, I changed how folders work too much
from IPython.display import HTML

directory = "/content/guildedchatexporter/(server)/(group)/(channel)/(channel).html" # @param {type:"string"}

with open(directory, "r") as f:
    html_content = f.read()
display(HTML(html_content))


In [ ]:
# @title DM Message Deleter
# @markdown A quick mass message deleter tool only for DMs. Doesn't do servers yet, I'll do it if someone begs like for how someone asked for v2.0 fixes as I figured deleted servers are good enough.

# @markdown Use channel_id if you want to do one-by-one (use Copy Channel ID or copy it from the browser URL), or use deleteAllDMs to nuke all DMs. The collection process can be slow for huge DMs too, be patient. This tool will not "close" the DMs once it's done.

import requests
import time

def del_collector(channel_id, page_size=100):
    messages = []
    beforeDate = None
    while True:
        gurl = f"https://www.guilded.gg/api/channels/{channel_id}/messages"
        params = {"maxItems": page_size}
        if beforeDate:
            params["beforeDate"] = beforeDate
        response = requests.get(gurl, params=params, cookies=cookies)
        if response.status_code != 200:
            print(f"Failed to fetch messages: {response.status_code}")
            break
        data = response.json()
        batch = data.get("messages", [])
        if not batch:
            break
        messages.extend(batch)
        if len(batch) < page_size:
            break
        beforeDate = batch[-1]["createdAt"]
    return messages

def del_preview(msg): # lazy
    try:
        return msg["content"]["document"]["nodes"][0]["nodes"][0]["leaves"][0]["text"]
    except Exception:
        return ""

def delete_message(channel_id, message_id):
    url = f"https://www.guilded.gg/api/channels/{channel_id}/messages/{message_id}"
    r = requests.delete(url, cookies=cookies)
    return r.status_code

def del_channel(channel_id):
    myself_data = fetch_myself()
    user_id = myself_data["user"]["id"]
    print(f"fetching messages")
    msgs = del_collector(channel_id)
    print(f"  {len(msgs)} messages total - {len([m for m in msgs if m.get('createdBy') == user_id])} by you")

    total = sum(1 for m in msgs if m.get('createdBy') == user_id) #lazy
    current = 0
    deleted = 0
    for m in msgs:
        if m.get('createdBy') == user_id:
            current += 1
            content = del_preview(m)
            content_short = (content[:61] + '...') if len(content) > 64 else content
            status = delete_message(channel_id, m['id'])
            if status in (200, 204):
                deleted += 1
                print(f"[{current}/{total}]: deleted {m['id']}: \"{content_short}\"")
            elif status == 429:
                print(f"rate limited, slowing down...")
                time.sleep(2)
                continue
            else:
                print(f"[{current}/{total}] Failed to delete {m['id']}: {status}")
            time.sleep(0.5)
    print(f"\nDeleted {deleted} in {channel_id}\nthis number may be off if there's a system msg or something, double check results")

channel_id = "AAAAAAAA-BBBB-CCCC-DDDD-EEEEEEEEEEEE" # @param {"type":"string"}
deleteAllDMs = False # @param {"type":"boolean"}

if deleteAllDMs:
    print("Starting to delete every DM you have, cancel this process now if you made a mistake")
    time.sleep(5)
    myself_data = fetch_myself()
    USER_ID = myself_data["user"]["id"]
    dms_data = fetch_dms()
    for channel in dms_data['channels']:
        channel_id = channel['id']
        print(f"\n---\nProcessing channel {channel_id}")
        del_channel(channel_id)
    print("\nDone with all DMs.")
else:
    del_channel(channel_id)
